In [24]:
import pandas as pd

df = pd.read_excel("processed/msamogh-turn-durations-seconds.xlsx")
df["turn_durations"] = df["turn_durations"].apply(eval)

In [25]:
df["even_turn_duration"] = df["turn_durations"].apply(lambda x: sum([v for k, v in x.items() if int(k) % 2 == 0]))
df["odd_turn_duration"] = df["turn_durations"].apply(lambda x: sum([v for k, v in x.items() if int(k) % 2 != 0]))
df.head()

,trajectory,prompt,turn_durations,even_turn_duration,odd_turn_duration
0,"GazePath(submit_time=1648243465776, dst={'area...",semi,"{'2': 14.796999454498291, '1': 5.4239993095397...",39.333999,10.252
1,"GazePath(submit_time=1648243136082, dst={'area...",semi,"{'0': 10.121999979019165, '4': 19.889999866485...",55.585999,29.145
2,"GazePath(submit_time=1648243035969, dst={'area...",semi,"{'4': 33.115999937057495, '0': 34.605999708175...",104.575000,21.390
3,"GazePath(submit_time=1648243317732, dst={'area...",semi,"{'0': 9.204999923706055, '1': 3.39199995994567...",29.280999,4.471
4,"GazePath(submit_time=1648243279922, dst={'area...",semi,"{'0': 14.164000034332275, '4': 9.2470004558563...",46.672001,24.226


In [26]:
df.groupby("prompt").agg({"even_turn_duration": "mean", "odd_turn_duration": "mean"})

,even_turn_duration,odd_turn_duration
prompt,,
goal,57.916403,18.751913
semi,51.338579,26.632474


In [27]:
df.groupby("prompt").agg({"even_turn_duration": "median", "odd_turn_duration": "median"})

,even_turn_duration,odd_turn_duration
prompt,,
goal,57.215001,14.681001
semi,51.918500,21.717500


Normalize turn durations

In [28]:
df["user_turn_duration_norm"] = df.apply(lambda x: x["even_turn_duration"] / (x["even_turn_duration"] + x["odd_turn_duration"]), axis=1)
df["sys_turn_duration_norm"] = df.apply(lambda x: x["odd_turn_duration"] / (x["even_turn_duration"] + x["odd_turn_duration"]), axis=1)

In [35]:
df.groupby("prompt").agg({"user_turn_duration_norm": ["mean", "std"], "sys_turn_duration_norm": ["mean", "std"]})

user_turn_duration_norm           sys_turn_duration_norm          
                          mean       std                   mean       std
prompt                                                                   
goal                  0.755804  0.178644               0.244196  0.178644
semi                  0.673766  0.193808               0.326234  0.193808

In [29]:
df.groupby("prompt").agg({"user_turn_duration_norm": "median", "sys_turn_duration_norm": "median"})

,user_turn_duration_norm,sys_turn_duration_norm
prompt,,
goal,0.789324,0.210676
semi,0.710191,0.289809


In [30]:
from scipy.stats import ttest_ind

In [39]:
ttest_ind(
    df[df["prompt"] == "goal"]["user_turn_duration_norm"],
    df[df["prompt"] == "semi"]["user_turn_duration_norm"],
)

Ttest_indResult(statistic=3.8165276221207356, pvalue=0.00016449803527817126)

In [ ]:
ttest_ind(df[df["prompt"] == "semi"]["sys_turn_duration_norm"], df[df["prompt"] == "goal"]["sys_turn_duration_norm"])

Ttest_indResult(statistic=3.816527622120729, pvalue=0.0001644980352781755)

In [37]:
df.columns

Index(['trajectory', 'prompt', 'turn_durations', 'even_turn_duration',
       'odd_turn_duration', 'user_turn_duration_norm',
       'sys_turn_duration_norm'],
      dtype='object')

In [38]:
len(df)

301